In [1]:
import requests

**200**: Everything went okay, and the result has been returned (if any).

**301**: The server is redirecting you to a different endpoint. This can happen when a company switches domain names, or an endpoint name is changed.

**400**: The server thinks you made a bad request. This happens when you send incorrect data or make other client-side errors.

**401**: The server thinks you're not authenticated. Many APIs require login credentials, so this happens when you don't send the right credentials to access an API.

**403**: The resource you're trying to access is forbidden: you don't have the right permissions to see it.

**404**: The resource you tried to access wasn't found on the server.

**503**: The server is not ready to handle the request.

In [14]:
import sys
import requests
import json
import platform
import pandas as pd
from io import StringIO

API_ENDPOINT_BASE_URL = "https://www.immport.org"
DATA_QUERY_URL = API_ENDPOINT_BASE_URL + "/data/query"
ASPERA_TOKEN_URL = API_ENDPOINT_BASE_URL + "/data/download/token"
IMMPORT_TOKEN_URL = "https://www.immport.org/auth/token"


def request_immport_token(immport_token_url, username, password):
    '''Request an ImmPort token

       :param username: ImmPort user name.
       :param password: ImmPort user password.

       return immport_token
    '''
    r = requests.post(immport_token_url,
                  data={'username': username, 'password': password})
    if r.status_code == 200:
        return r.json()['access_token']
    else:
        return None


def api_data_query(username, password, endpoint, immport_token_url, token=True, format="json"):
    '''Use the Data Query API by first checking ImmPort user credentials,
       retrieving an ImmPort token, then call the API endpoint to retrieve the results.

       param user_name: ImmPort username.
       param password: ImmPort password.
       param endpoint: Data Query endpoint
       param token: Indicates if endpoint requires a token
       param format: String (json, tsv)


       return: results or None
    '''
    headers = {}
    results = None

    if token:
        immport_token = request_immport_token(immport_token_url, username, password)
        if immport_token is None:
             print("ERROR: Credentials incorrect for ImmPort, unable to retrieve token", file = sys.stderr)
             return None
        if format == "json":
            headers = {
                'Authorization': "bearer " + immport_token,
                'Content-Type': "application/json"
            }
        else:
            headers = {
                'Authorization': "bearer " + immport_token,
                'Content-Type': "text/plain"
            }

    else:
        if format == "json":
            headers = {
                'Content-Type': "application/json"
            }
        else:
            headers = {
                'Content-Type': "text/plain"
            }

    results = requests.get(endpoint, headers=headers)
    if results.status_code == 200:
        if results is None:
             print("ERROR: API Endpoint failed to return results", file = sys.stderr)
             return None
        if format == "json":
            return results.json()
        else:
            return results.text
    else:
        print("ERROR: API Status Code: " + str(results.status_code), file = sys.stderr)
        return None

? is a delimiter that marks the beginning of the query string. It separates the path of the URL from the parameters that are being passed.

filter_by indicates the type of filtering we are applying.
region is a specific field in the API's database that we want to filter by. In this context, region refers to the geographical area of the countries.

The first = sign following filter_by is used to assign the filtering criteria (region in this case), and the second = sign assigns the specific value (Sub-Saharan Africa) to the region field.

%20 is URL encoding for a space character, necessary because URLs cannot contain actual space characters. However, when composing a GET request in an editor or a tool, you don't need to manually type %20 for spaces; it is typically handled automatically by the software.

In [21]:
USERNAME = 'aleliu'
PASSWORD = 'NiNi!^**9'

In [22]:
endpoint = DATA_QUERY_URL + "/api/study/pubmed/SDY1?format=json"
results = api_data_query(USERNAME, PASSWORD, endpoint, IMMPORT_TOKEN_URL, True, "json" )
df = pd.read_json(json.dumps(results))
df.head(2)

/var/folders/z6/4h2s7dds6rx9wjn6fhrrgzw9wrk0b4/T/ipykernel_68208/2162617731.py:3: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json.dumps(results))


,pubmedId,studyAccession,authors,doi,issue,journal,month,pages,title,year
0,16387596,SDY1,"Casale TB, Busse WW, Kline JN, Ballas ZK, Moss...",10.1016/j.jaci.2005.09.036,117(1),The Journal of allergy and clinical immunology,Jan,134-40,Omalizumab pretreatment decreases acute reacti...,2006
1,17631952,SDY1,"Klunker S, Saggar LR, Seyfert-Margolis V, Asar...",10.1016/j.jaci.2007.05.034,120(3),The Journal of allergy and clinical immunology,Sep,688-95,Combination treatment with omalizumab and rush...,2007
